## Step 1: Import the necessary libraries:


In [158]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
import itertools
import ast
import matplotlib.pyplot as plt 
import os

## Step 2: Define the environment:


In [159]:
devices_path = "./resources/devices.csv"
tasks_path = "./resources/tasks.csv"
dataFile = "./results/power.csv"

#### ALL THE DEVICES


In [160]:
devices = pd.read_csv(devices_path)

devices["voltages_frequencies"] = devices["voltages_frequencies"].apply(lambda x: ast.literal_eval(x))
devices["capacitance"] = devices["capacitance"].apply(
    lambda x: ast.literal_eval(x)
)
devices["occupied_cores"] = devices["occupied_cores"].apply(
    lambda x: ast.literal_eval(x)
)
devices["powerIdle"] = devices["powerIdle"].apply(
    lambda x: ast.literal_eval(x)
)
devices["acceptableTasks"] = devices["acceptableTasks"].apply(
    lambda x: ast.literal_eval(x)
)
devices = devices.drop(["Unnamed: 0"],axis=1)

### Step 2.2: Application


#### _ALL THE TASKS_


In [161]:
def read_tasks():
    tasks = pd.read_csv(tasks_path)
    tasks = tasks.sample(frac=1)
    tasks = tasks.set_index("id")
    tasks_copy = tasks.copy()
    tasks_copy = tasks_copy.drop(["job","dependency","mobility","status"],axis=1)
    taskList = tasks_copy.index.tolist()
    return taskList, tasks_copy

## Step 4 : DDT


### Step 4.1: Initializing The tree


In [162]:
class DDT(nn.Module):
    def __init__(self, num_input, num_output, depth, max_depth):
        super(DDT, self).__init__()
        self.depth = depth
        self.max_depth = max_depth
        self.exploration_rate = 0.10
        if depth != max_depth:
            self.weights = nn.Parameter(torch.empty(num_input).normal_(mean=0, std=0.1))
            self.bias = nn.Parameter(torch.zeros(1))
            self.alpha = nn.Parameter(torch.zeros(1))
        if depth == max_depth:
            self.prob_dist = nn.Parameter(torch.zeros(num_output))
        if depth < max_depth:
            self.left = DDT(num_input, num_output, depth + 1, max_depth)
            self.right = DDT(num_input, num_output, depth + 1, max_depth)

    def forward(self, x):
        if self.depth == self.max_depth:
            return self.prob_dist
        val = torch.sigmoid(self.alpha * (torch.matmul(x, self.weights.t()) + self.bias))
        if val >= 0.5:
            return val * self.right(x)
        else:

            return (1 - val) * self.left(x)

## Step 5: RL


In [163]:
def calc_execution_time(device, task, core, dvfs):
    if device['id'] == "cloud":
        return task["computationalLoad"] / device["voltages_frequencies"][0]
    else:
        return task["computationalLoad"] / device["voltages_frequencies"][core][dvfs][0]


def calc_power_consumption(device, task, core, dvfs):
    if device['id'] == "cloud":return 13.85 
    return (device["capacitance"][core]* (device["voltages_frequencies"][core][dvfs][1] ** 2)* device["voltages_frequencies"][core][dvfs][0])
def calc_energy(device, task, core, dvfs):
    return calc_execution_time(device, task, core, dvfs) * calc_power_consumption(device, task, core, dvfs)


def calc_total(device, task, core, dvfs):
    timeTransMec = 0
    timeTransCC = 0
    exeTime = 0

    transferRate5g =1e9
    latency5g=5e-3
    transferRateFiber =1e10
    latencyFiber=1e-3

    timeDownMec = task["returnDataSize"] / transferRate5g
    timeDownMec += latency5g
    timeUpMec = task["dataEntrySize"] / transferRate5g
    timeUpMec += latency5g

    alpha = 52e-5
    beta = 3.86412
    powerMec = alpha * 1e9 / 1e6 + beta

    timeDownCC = task["returnDataSize"] / transferRateFiber
    timeDownCC += latencyFiber
    timeUpCC = task["dataEntrySize"] / transferRateFiber
    timeUpCC += latencyFiber

    powerCC = 3.65 


    if device["id"].startswith("mec"):
        timeTransMec =  timeUpMec +  timeDownMec 
        exeTime = calc_execution_time(device, task, core, dvfs)
        totalTime = exeTime + timeTransMec
        p = calc_power_consumption(device, task, core, dvfs) 
        totalPower = p + powerMec
        

    elif device['id'].startswith("cloud"):
        timeTransMec =  timeUpMec +  timeDownMec 
        timeTransCC = timeUpCC+timeDownCC
        exeTime = calc_execution_time(device, task, core, dvfs)
        totalTime =  exeTime + timeTransMec +timeTransCC
        p = calc_power_consumption(device, task, core, dvfs)
        totalPower = p + powerCC + powerMec

        

    elif device['id'].startswith("iot"):
        exeTime = calc_execution_time(device, task, core, dvfs)
        totalTime = exeTime
        p = calc_power_consumption(device, task, core, dvfs)
        totalPower = p

    return totalTime , totalPower

In [164]:
def checkIfSuitable(state, device):
    safeFail = 0
    taskFail = 0
    if  state['safe'] and not device["handleSafeTask"]:
        safeFail = 1
    if state['kind'] not in device["acceptableTasks"]:
        taskFail = 1
    return taskFail,safeFail

In [165]:
def plot_histories(rSetup, punish, learning_mode, lossHistory, avg_time_history, avg_energy_history, avg_fail_history):
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))  # Create a grid of 2x2 for plots

    # Set a comprehensive title for the figure with dynamic parameters
    plt.suptitle(f"Training History with setup {rSetup}, punish: {punish}, mode: {learning_mode}", fontsize=16, fontweight='bold')

    # Plot for average loss history
    loss_values = [l.detach().numpy() if isinstance(l, torch.Tensor) else l for l in lossHistory]
    axs[0, 0].plot(loss_values, label='Average Loss', color='blue', marker='o')  # Add markers for clarity
    axs[0, 0].set_title('Average Loss History')
    axs[0, 0].set_xlabel('Epochs')
    axs[0, 0].set_ylabel('Loss')
    axs[0, 0].legend()
    axs[0, 0].grid(True)

    # Plot for average time history
    time_values = np.array(avg_time_history)  # Ensure data is in numpy array
    axs[0, 1].plot(time_values, label='Average Time', color='red', marker='o')
    axs[0, 1].set_title('Average Time History')
    axs[0, 1].set_xlabel('Epochs')
    axs[0, 1].set_ylabel('Time')
    axs[0, 1].legend()
    axs[0, 1].grid(True)

    # Plot for average energy history
    energy_values = np.array(avg_energy_history)
    axs[1, 0].plot(energy_values, label='Average Power', color='green', marker='o')
    axs[1, 0].set_title('Average Power History')
    axs[1, 0].set_xlabel('Epochs')
    axs[1, 0].set_ylabel('Power')
    axs[1, 0].legend()
    axs[1, 0].grid(True)

    # Plot for average fail history
    fail_values = np.array(avg_fail_history)
    axs[1, 1].plot(fail_values, label='Average Fail', color='purple', marker='o')
    axs[1, 1].set_title('Average Fail History')
    axs[1, 1].set_xlabel('Epochs')
    axs[1, 1].set_ylabel('Fail Count')
    axs[1, 1].legend()
    axs[1, 1].grid(True)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) 
    # plt.show() # Adjust layout to prevent overlap, leaving space for the title
    plt.savefig(f"./results/Power Figs/r{rSetup}_p{punish}_m{learning_mode}")

In [166]:
def getSetup(p, t, setup, alpha=10, beta=5):
    match setup:
        case 1:
            return  -1 * (alpha * p + beta * t)
        case 2:
            return  -1 / (alpha * p + beta * t)
        case 3:
            return  -np.exp(alpha * p) - np.exp(beta * t)
        case 4:
            return  -np.exp(alpha * p + beta * t)
        case 5:
            return  np.exp(-1 * (alpha * p + beta * t))
        case 6:
            return  np.log(alpha * p + beta * t)
        case 7: 
            return  -((alpha * p + beta * t) ** 2)

In [167]:
class Environment:
    def __init__(self):
        self.learning_mode = 0
        self.rewardSetup = 1
        self.punish = 0
        self.alpha = 1
        self.beta = 1

        self.taskList = []
        self.tasks_copy = None

        self.feature_size = 5
        self.num_actions = len(devices)
        self.max_depth = 3
        self.agent = DDT(self.feature_size, self.num_actions,depth=0, max_depth=self.max_depth)
        self.optimizer = optim.Adam(self.agent.parameters(), lr=0.005)

        self.avg_time_history = np.array([])
        self.avg_energy_history = np.array([])
        self.avg_fail_history = np.array([0,0,0])
        self.avg_loss_history = np.array([])
        self.avg_reward_history = np.array([])
     
    def execute_action(self, state, action):
        self.taskList.pop(0)
        device = devices.iloc[action]        
        taskFail, safeFail = checkIfSuitable(state, device)
        
        if not (taskFail or safeFail):
            for coreIndex in range(len(device["occupied_cores"])):
                if device["occupied_cores"][coreIndex] == 0:
                    total_t, total_p  = calc_total(device, state, coreIndex,0)

                    reward = getSetup(total_p ,total_t, self.rewardSetup)
                    return (reward,total_t,total_p,0,0)
                
        return (self.punish,0,0, taskFail,safeFail)


    def train(self, num_epoch, num_episodes):
        total_avg_t = 0
        total_avg_e = 0
        total_avg_r = 0
        total_avg_l = 0
        total_avg_fail = np.array([0,0,0],dtype=np.float64)
        
        for i in range(num_epoch):
            total_fail_epoch = np.array([0,0,0])
            total_reward_epoch = 0
            total_loss_epoch = 0
            total_time_epoch = 0
            total_energy_epoch = 0

            
            for j in range(num_episodes):
                state = self.tasks_copy.loc[self.taskList[0]]
                x = torch.tensor(np.array(state.values, dtype=np.float32)).unsqueeze(0)
                
                output = self.agent(x)
                action_probabilities = torch.softmax(output, dim=0)
                action_index = torch.multinomial(action_probabilities, 1).item()

                reward, t, e,taskFail,safeFail = self.execute_action(state, action_index)
                loss = (output[action_index] * reward)

                #single reward:
                if self.learning_mode == 0:
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()
                    
                total_reward_epoch += reward
                total_loss_epoch += loss
                total_time_epoch += t
                total_energy_epoch += e
                fails = np.array([taskFail + safeFail, taskFail, safeFail])
                total_fail_epoch += fails
                
            #total loss
            if self.learning_mode == 1:
                self.optimizer.zero_grad()
                total_loss_epoch.backward()
                self.optimizer.step()   
            
            avg_time = total_time_epoch / num_episodes
            avg_energy = total_energy_epoch / num_episodes
            avg_reward = total_reward_epoch / num_episodes
            avg_loss = total_loss_epoch/num_episodes
            avg_fail = [elem/num_episodes for elem in total_fail_epoch]


            #avg reward
            if self.learning_mode == 2:
                self.optimizer.zero_grad()
                avg_loss.backward()
                self.optimizer.step()
            
            
            self.avg_loss_history = np.append(self.avg_loss_history,avg_loss.detach().numpy())
            self.avg_reward_history = np.append(self.avg_reward_history,avg_reward)
            self.avg_time_history = np.append(self.avg_time_history,avg_time)
            self.avg_energy_history = np.append(self.avg_energy_history,avg_energy)
            self.avg_fail_history = np.vstack([self.avg_fail_history,avg_fail])



            total_avg_t += avg_time
            total_avg_e += avg_energy
            total_avg_l += avg_loss
            total_avg_r += avg_reward
            total_avg_fail += avg_fail
            
            

        avg_avg_t = total_avg_t / num_epoch
        avg_avg_l = total_avg_l / num_epoch
        avg_avg_r = total_avg_r / num_epoch
        avg_avg_e = total_avg_e / num_epoch



        half_num_epoch = num_epoch//2

        new_epoch_data = {
            "Setup": self.rewardSetup,
            "Learning Mode": self.learning_mode,
            "Punishment": self.punish,
            "Alpha": self.alpha,
            "Beta": self.beta,

            "Average Loss": avg_avg_l.detach(),
            "Last Epoch Loss": self.avg_loss_history[-1],

            "Task Converge": np.argmax(np.flip(self.avg_fail_history[:, 1])!= 0),
            "Task Fail Percentage": np.count_nonzero(self.avg_fail_history[:, 1])/len(self.avg_fail_history[:, 1]),
            "Safe Converge":np.argmax(np.flip(self.avg_fail_history[:, 2])!= 0),
            "Safe Fail Percentage": np.count_nonzero(self.avg_fail_history[:, 2])/len(self.avg_fail_history[:, 2]),
           
            "Average Time": avg_avg_t,
            "Last Epoch Time": self.avg_time_history[-1],
           
            "Average Energy": avg_avg_e,
            "Last Epoch Energy":  self.avg_energy_history[-1],
           
            "Average Reward": avg_avg_r,
            "Last Epoch Reward": self.avg_reward_history[-1],

            "First 10 Avg Time": np.mean(self.avg_time_history[:10]),
            "Mid 10 Avg Time": np.mean(self.avg_time_history[half_num_epoch:half_num_epoch + 10]),
            "Last 10 Avg Time": np.mean(self.avg_time_history[:-10]),
            
            "First 10 Avg Energy":np.mean(self.avg_energy_history[:10]),
            "Mid 10 Avg Energy":np.mean(self.avg_energy_history[half_num_epoch:half_num_epoch + 10]),
            "Last 10 Avg Energy":np.mean(self.avg_energy_history[:-10]),  

            "First 10 Avg Reward":np.mean(self.avg_reward_history[:10]),
            "Mid 10 Avg Reward":np.mean(self.avg_reward_history[half_num_epoch:half_num_epoch + 10]),
            "Last 10 Avg Reward":np.mean(self.avg_reward_history[:-10]),


            "First 10 Avg Loss":np.mean(self.avg_loss_history[:10]),
            "Mid 10 Avg Loss":np.mean(self.avg_loss_history[half_num_epoch:half_num_epoch + 10]),
            "Last 10 Avg Loss":np.mean(self.avg_loss_history[:-10]),
            
            "First 10 (total, task, safe) Fail": np.mean(self.avg_fail_history[:10],axis=0),
            "Mid 10 (total, task, safe) Fail": np.mean(self.avg_fail_history[half_num_epoch:half_num_epoch + 10],axis=0),
            "Last 10 (total, task, safe) Fail":np.mean(self.avg_fail_history[:-10],axis=0),
        }   


        df = None
        if  os.path.exists(dataFile):
            df = pd.read_csv(dataFile)

            # Convert the new data into a DataFrame and concatenate it
            new_df = pd.DataFrame(new_epoch_data)
            df = pd.concat([df, new_df], ignore_index=True)

            # Save the updated DataFrame back to CSV
        else: 
            df = pd.DataFrame(new_epoch_data)

        df.to_csv(dataFile, index=False)

        plot_histories(self.rewardSetup,self.punish,self.learning_mode,self.avg_loss_history,self.avg_time_history,self.avg_energy_history,self.avg_fail_history[:, 0])




In [168]:

# Define the column headers as a list
headers = [
    "Setup","Learning Mode", "Punishment", "Alpha", "Beta","Average Loss", "Last Epoch Loss", "Task Converge", "Task Fail Percentage", "Safe Converge", "Safe Fail Percentage", "Average Time","Last Epoch Time",
    "Average Energy", "Last Epoch Energy", "Average Reward", "Last Epoch Reward", "First 10 Avg Time", "Mid 10 Avg Time","Last 10 Avg Time","First 10 Avg Energy", "Mid 10 Avg Energy","Last 10 Avg Energy", "First 10 Avg Reward",  "Mid 10 Avg Reward", "Last 10 Avg Reward", "First 10 Avg Loss", "Mid 10 Avg Loss", "Last 10 Avg Loss","First 10 (total, task, safe) Fail", "Mid 10 (total, task, safe) Fail", "Last 10 (total, task, safe) Fail"
]
# Create an empty DataFrame with these headers
df = pd.DataFrame(columns=headers)

# Specify the filename
filename = dataFile

# Save the DataFrame to a CSV file
df.to_csv(filename, index=False)

print(f"CSV file '{filename}' created successfully with headers only.")

In [169]:

def train_test(n):
    rpSetup_list = {1: [10, 100, 1000]}
    learning_modes = [0 , 1, 2]
    for reward, punishments in rpSetup_list.items():
        for punish, mode in itertools.product(punishments, learning_modes): 
            taskList, tasks_copy = read_tasks()
            env = Environment()
            env.learning_mode = mode
            env.rewardSetup = reward
            env.punish = punish
            env.taskList = taskList
            env.tasks_copy = tasks_copy
            
            tree = env.agent
            env.train(10001, 10)

train_test(1)   
print("completed")
